# Fire analysis script
The goal of this script is to complete a regression where region of interest type, saliency, and other features are used to predict fixation behavior.

## 1. Load eye movement data

This cell loads your raw eye movement data from a CSV file.

In [3]:
import pandas as pd

# Update the filename if needed
eye_df = pd.read_csv("RawData_AllAnimals.csv")
eye_df.head()

,Subject,Stimuli,FixStart,FixEnd,FixDur,XPos,YPos,ROI,Block,Trial,ImageType,Species,SubjectName
0,Cheyenne_20240702_1420,FStil09.png,75.961,206.049,133.330,1146.815,1358.280,OffScreen,1,Cheyenne_Block1_20240702_1420.xlsx,F,Baboon,Cheyenne
1,Cheyenne_20240702_1420,FStil09.png,232.713,392.695,163.256,1527.230,561.030,NonFire,1,Cheyenne_Block1_20240702_1420.xlsx,F,Baboon,Cheyenne
2,Cheyenne_20240702_1420,FStil09.png,409.259,645.983,239.965,1093.810,378.755,Fire,1,Cheyenne_Block1_20240702_1420.xlsx,F,Baboon,Cheyenne
3,Cheyenne_20240702_1420,FStil09.png,655.942,869.192,216.689,1193.255,525.905,NonFire,1,Cheyenne_Block1_20240702_1420.xlsx,F,Baboon,Cheyenne
4,Cheyenne_20240702_1420,FStil09.png,899.287,1055.843,159.995,1664.420,123.980,NonFire,1,Cheyenne_Block1_20240702_1420.xlsx,F,Baboon,Cheyenne


## 2. Load ROI masks and saliency maps

This cell loads all ROI type/index masks and saliency maps into dictionaries for fast lookup by image name.

In [5]:
import numpy as np
import os

# Helper to get base image name from Stimuli column
def get_base_name(stimuli):
    return os.path.splitext(os.path.basename(stimuli))[0]

# Load ROI masks
roi_type_dict = {}
roi_index_dict = {}

for fname in os.listdir("contour_masks"):
    if "roi_type" in fname:
        key = fname.split("_roi_type")[0]
        roi_type_dict[key] = np.load(os.path.join("contour_masks", fname))
    elif "roi_index" in fname:
        key = fname.split("_roi_index")[0]
        roi_index_dict[key] = np.load(os.path.join("contour_masks", fname))

# Load saliency maps robustly
saliency_dict = {}
for fname in os.listdir("csv_output"):
    if fname.endswith("_saliency.npy"):
        key = fname.split("_saliency")[0]
        saliency_dict[key] = np.load(os.path.join("csv_output", fname))
    elif fname.endswith("_saliency.csv"):
        key = fname.split("_saliency")[0]
        # Try reading as CSV, skip header if present
        try:
            arr = np.loadtxt(os.path.join("csv_output", fname), delimiter=',')
        except Exception:
            arr = pd.read_csv(os.path.join("csv_output", fname), header=None).values
        saliency_dict[key] = arr

## 3. Join eye movement data with image features

This cell creates a new DataFrame where each fixation is enriched with ROI type, ROI index, and saliency at the fixation location.

In [6]:
features = []

for idx, row in eye_df.iterrows():
    img_key = get_base_name(row['Stimuli'])
    x, y = int(row['XPos']), int(row['YPos'])
    
    # Skip if image not found or fixation out of bounds
    if img_key not in roi_type_dict or img_key not in saliency_dict:
        continue
    roi_type = roi_type_dict[img_key]
    roi_index = roi_index_dict[img_key]
    saliency = saliency_dict[img_key]
    if y >= roi_type.shape[0] or x >= roi_type.shape[1]:
        continue

    features.append({
        "Subject": row["Subject"],
        "Stimuli": row["Stimuli"],
        "FixStart": row["FixStart"],
        "FixEnd": row["FixEnd"],
        "FixDur": row["FixDur"],
        "XPos": x,
        "YPos": y,
        "ROI": row["ROI"],
        "Block": row["Block"],
        "Trial": row["Trial"],
        "ImageType": row["ImageType"],
        "Species": row["Species"],
        "SubjectName": row["SubjectName"],
        "roi_type": roi_type[y, x],
        "roi_index": roi_index[y, x],
        "saliency": saliency[y, x]
    })

fixation_features_df = pd.DataFrame(features)
fixation_features_df.head()

""


## 4. Example regression: Predicting fire region fixations

This cell shows how to use the joined DataFrame to predict whether a fixation falls on a fire region (roi_type == 1) using saliency and fixation duration as predictors.

In [11]:
print(fixation_features_df.shape)
print(fixation_features_df.columns)

print(eye_df['Stimuli'].head())
print([get_base_name(s) for s in eye_df['Stimuli'].head()])
print(list(roi_type_dict.keys())[:5])
print(list(saliency_dict.keys())[:5])

(0, 0)
Index([], dtype='object')
0    FStil09.png
1    FStil09.png
2    FStil09.png
3    FStil09.png
4    FStil09.png
Name: Stimuli, dtype: object
['FStil09', 'FStil09', 'FStil09', 'FStil09', 'FStil09']
['FStil14.jpg', 'FStil21.jpg', 'FStil04.jpg', 'FStil25.jpg', 'FStil10.jpg']
['FStil17.jpg', 'FStil08.jpg', 'FStil22.jpg', 'FStil03.jpg', 'FStil07.jpg']


In [10]:
from sklearn.linear_model import LogisticRegression

# Create binary target: 1 if fire region, 0 otherwise
fixation_features_df["is_fire"] = (fixation_features_df["roi_type"] == 1).astype(int)
X = fixation_features_df[["saliency", "FixDur"]]
y = fixation_features_df["is_fire"]

model = LogisticRegression()
model.fit(X, y)

print("Regression coefficients:", model.coef_)

KeyError: 'roi_type'

In [ ]:
#import numpy as np
#import os
#import pandas as pd
#
#contour_masks = os.listdir("contour_masks")
#roi_index_filenames = [f for f in contour_masks if "roi_index" in f]
#roi_type_filenames = [f for f in contour_masks if "roi_type" in f]
#
## Load the roi_index and roi_type files
#roi_index_list = []
#roi_type_list = []
#
##loop through the roi_index and roi_type files and load them into lists
#for filename in roi_index_filenames:
#    roi_index = np.load(os.path.join("contour_masks", filename))
#    roi_index = roi_index.astype(np.uint8)
#    roi_index_list.append((filename, roi_index))
#
#for filename in roi_type_filenames:
#    roi_type = np.load(os.path.join("contour_masks", filename))
#    roi_type = roi_type.astype(np.uint8)
#    roi_type_list.append((filename, roi_type))
#
## Convert lists to numpy arrays
#roi_index_combined = np.array(roi_index_list, dtype=object)
#roi_type_combined = np.array(roi_type_list, dtype=object)